In [ ]:
import socket
import threading

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 7002        # Port to listen on (non-privileged ports are > 1023)

clients = []
client_lock = threading.Lock()  # Lock for thread-safe client list management

def handle_client(client_socket, address):
    """Handles communication with a single connected client in a separate thread."""
    print(f"Connected by {address}")
    while True:
        try:
            data = client_socket.recv(1024).decode('utf-8')
            if not data:
                break

            # Broadcast message to all connected clients (excluding sender)
            with client_lock:
                for client in clients:
                    if client[0] != client_socket:
                        client[0].sendall(f"[Server] {data}\n".encode('utf-8'))

            print(f"Client {address}: {data}")
        except ConnectionError:
            print(f"Client {address} disconnected")
            break

    with client_lock:
        clients.remove((client_socket, address))
    client_socket.close()

def main():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind((HOST, PORT))
    server_socket.listen(5)

    print(f"Server listening on {HOST}:{PORT}")

    while True:
        client_socket, address = server_socket.accept()
        client_thread = threading.Thread(target=handle_client, args=(client_socket, address))
        client_thread.start()

        with client_lock:
            clients.append((client_socket, address))

if __name__ == "__main__":
    main()


In [ ]:
import socket
import threading
import time

HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 7002        # The port used by the server

def receive_messages(client_socket):
    """Continuously receives messages from the server and displays them."""
    while True:
        try:
            data = client_socket.recv(1024).decode('utf-8')
            if not data:
                break
            print(data)
        except ConnectionError:
            print("Server disconnected")
            break

def send_messages(client_socket):
    """Continuously sends messages from the user to the server."""
    while True:
        message = input()
        client_socket.sendall(message.encode('utf-8'))
        time.sleep(0.1)  # Small delay to avoid excessive sending

def main():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((HOST, PORT))

    receive_thread = threading.Thread(target=receive_messages, args=(client_socket,))
    receive_thread.daemon = True  # Set receiving as background thread
    receive_thread.start()

    send_messages(client_socket)

if __name__ == "__main__":
    main()


In [ ]:
import socket
import threading
import tkinter as tk

class ChatServer:
    def __init__(self, host, port):
        # Initialize server parameters and data structures
        self.HOST = host
        self.PORT = port
        self.clients = []  # List to store connected clients (socket, address)
        self.client_lock = threading.Lock()  # Lock to synchronize access to clients list

        # Create a server socket and set options
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.server_socket.bind((self.HOST, self.PORT))
        self.server_socket.listen(5)

        # Create a tkinter window for the server GUI
        self.server_window = tk.Tk()
        self.server_window.title("Chat Server")

        # GUI elements: labels, listbox, and text display area
        self.connected_clients_label = tk.Label(self.server_window, text="Connected Clients:")
        self.connected_clients_label.pack()

        self.connected_clients_list = tk.Listbox(self.server_window, width=50)
        self.connected_clients_list.pack()

        self.message_display_area = tk.Text(self.server_window, width=70, height=20)
        self.message_display_area.pack()

        # Configure window close event
        self.server_window.protocol("WM_DELETE_WINDOW", self.on_closing)

        # Start the server thread to listen for clients
        self.server_thread = threading.Thread(target=self.listen_for_clients)
        self.server_thread.start()

        # Start the tkinter main loop
        self.server_window.mainloop()

    def listen_for_clients(self):
        # Server listens for incoming client connections
        print(f"Server listening on {self.HOST}:{self.PORT}")
        while True:
            client_socket, address = self.server_socket.accept()
            client_thread = threading.Thread(target=self.handle_client, args=(client_socket, address))
            client_thread.start()

            # Add the new client to the clients list and update GUI
            with self.client_lock:
                self.clients.append((client_socket, address))
                self.update_connected_clients_list()

    def handle_client(self, client_socket, address):
        # Handle communication with a specific client
        print(f"Connected by {address}")
        while True:
            try:
                # Receive data from the client
                data = client_socket.recv(1024).decode('utf-8')
                if not data:
                    break

                # Broadcast the received message to all connected clients
                self.broadcast_message(f"[Client {address[0]}]: {data}")
                
                # Display the message in the server window
                self.display_message(f"[Client {address[0]}]: {data}")
            except ConnectionError:
                print(f"Client {address} disconnected")
                break

            # Remove the disconnected client from the list and update GUI
            with self.client_lock:
                self.clients.remove((client_socket, address))
                self.update_connected_clients_list()

        # Close the client socket when the client disconnects
        client_socket.close()

    def broadcast_message(self, message):
        # Send a message to all connected clients
        with self.client_lock:
            for client_socket, _ in self.clients:
                try:
                    client_socket.sendall(message.encode('utf-8'))
                except ConnectionError:
                    print(f"Error sending to client {client_socket}")

    def update_connected_clients_list(self):
        # Update the connected clients list in the GUI
        with self.client_lock:
            self.connected_clients_list.delete(0, tk.END)  # Clear existing list
            for _, address in self.clients:
                self.connected_clients_list.insert(tk.END, address[0])  # Update with client IP addresses

    def display_message(self, message):
        # Display a message in the server window
        self.message_display_area.insert(tk.END, message + "\n")
        self.message_display_area.see(tk.END)  # Scroll to the bottom

    def on_closing(self):
        # Handle server shutdown when the window is closed
        print("Shutting down server...")
        self.server_socket.close()
        for client_socket, _ in self.clients:
            try:
                client_socket.close()
            except ConnectionError:
                pass
        self.server_window.quit()

if __name__ == "__main__":
    HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
    
    # Create an instance of the ChatServer class with specified host and port
    server = ChatServer(HOST, 5555)
